In [6]:
from bs4 import BeautifulSoup
import requests
import json
import re
import time

def extract_medals_by_nation(sport_url, pays_dict):

    try:

        response = requests.get(sport_url)
        soup = BeautifulSoup(response.text, 'html.parser')
        

        sport_name = "Unknown Sport"
        sport_title = soup.select_one('h1')
        if sport_title:
            sport_name = sport_title.text.strip()
            sport_name = sport_name.replace("Olympic Medals in Sport ", "")
        else:

            match = re.search(r'/olympic-sport/([^/]+)', sport_url)
            if match:
                sport_name = match.group(1).replace('-', ' ')
        
        # Trouver les cartes de nations dans la section TOP NATIONS
        nations = []
        
        # Chercher directement les cartes de nations qui sont visibles
        nation_cards = soup.select('div.cards div.card.nation.visible')
        
        if not nation_cards:
            # Si pas trouvé avec cette méthode, essayer une autre
            nation_cards = soup.select('div.deck div.card.nation.visible')
        
        print(f"Nombre de nations trouvées pour {sport_name}: {len(nation_cards)}")
        
        for nation_card in nation_cards:
            # Extraire le nom du pays
            nation_name = ""
            name_elem = nation_card.select_one('div.bez')
            if name_elem:
                nation_name = name_elem.text.strip()
            
            # Si pas de nom, passer à la carte suivante
            if not nation_name:
                continue
            
            # Extraire le nombre total de médailles
            total_medals = 0
            medals_elem = nation_card.select_one('div.mal-ges')
            if medals_elem:
                medals_text = medals_elem.text.strip()
                medals_match = re.search(r'(\d+)', medals_text)
                if medals_match:
                    total_medals = int(medals_match.group(1))
            
            # Extraire les médailles d'or, d'argent et de bronze
            gold = 0
            silver = 0
            bronze = 0
            
            # Récupérer les médailles spécifiques
            medal_elems = nation_card.select('div.medals div[data-c]')
            for medal_elem in medal_elems:
                medal_count_elem = medal_elem.select_one('span.mal')
                if medal_count_elem:
                    medal_count = medal_count_elem.text.strip()
                    medal_count = int(medal_count) if medal_count.isdigit() else 0
                    
                    medal_type_elem = medal_elem.select_one('div.the-medal')
                    if medal_type_elem:
                        medal_type = medal_type_elem.get('data-medal')
                        if medal_type == '1':
                            gold = medal_count
                        elif medal_type == '2':
                            silver = medal_count
                        elif medal_type == '3':
                            bronze = medal_count
            
            # Créer l'objet nation
            nation = {
                "nom": nation_name,
                "medailles": {
                    "total": total_medals,
                    "or": gold,
                    "argent": silver,
                    "bronze": bronze
                }
            }
            
            # Ajouter le code pays si disponible
            for code, name in pays_dict.items():
                if name.lower() == nation_name.lower():
                    nation["code_pays"] = code
                    break
            
            nations.append(nation)
        
        return {
            "sport": sport_name,
            "nations": nations
        }
        
    except Exception as e:
        print(f"Erreur lors de l'extraction des médailles pour {sport_url}: {str(e)}")
        return {"sport": sport_url, "nations": [], "error": str(e)}

def main():
    # Dictionnaire des codes pays
    pays_dict = {
        'us': 'United States',
        'fr': 'France',
        'fi': 'Finland',
        'de': 'Germany',
        'it': 'Italy',
        'gb': 'Great Britain',
        'uk': 'United Kingdom',
        'ca': 'Canada',
        'au': 'Australia',
        'es': 'Spain',
        'se': 'Sweden',
        'ch': 'Switzerland',
        'no': 'Norway',
        'nl': 'Netherlands',
        'br': 'Brazil',
        'cn': 'China',
        'jp': 'Japan',
        'kr': 'South Korea',
        'ru': 'Russia',
        'ua': 'Ukraine',
        'pl': 'Poland',
        'ro': 'Romania',
        'cz': 'Czech Republic',
        'at': 'Austria',
        'hu': 'Hungary',
        'gr': 'Greece',
        'dk': 'Denmark',
        'be': 'Belgium',
        'mx': 'Mexico',
        'ar': 'Argentina',
        'za': 'South Africa',
        'nz': 'New Zealand',
        'tr': 'Turkey',
        'in': 'India',
        'pt': 'Portugal',
        'il': 'Israel',
        'ie': 'Ireland',
        'hr': 'Croatia',
        'bg': 'Bulgaria',
        'sk': 'Slovakia',
        'si': 'Slovenia',
        'th': 'Thailand',
        'sg': 'Singapore',
        'my': 'Malaysia',
        'id': 'Indonesia',
        'vn': 'Vietnam',
        'ph': 'Philippines',
        'ir': 'Iran',
        'eg': 'Egypt',
        'ke': 'Kenya',
        'et': 'Ethiopia',
        'ng': 'Nigeria',
        'co': 'Colombia',
        'cl': 'Chile',
        'pe': 'Peru',
        've': 'Venezuela',
        'lt': 'Lithuania',
        'iq': 'Iraq',
        'cu': 'Cuba',
        'pk': 'Pakistan',
        'by': 'Belarus',
        'ht': 'Haiti',
        'is': 'Iceland',
        'kz': 'Kazakhstan',
        'az': 'Azerbaijan',
        'jm': 'Jamaica',
        'do': 'Dominican Republic',
        'gh': 'Ghana',
        'rs': 'Serbia',
        'tt': 'Trinidad and Tobago',
        'lv': 'Latvia',
        'gy': 'Guyana',
        'am': 'Armenia',
        'bw': 'Botswana',
        'urs': 'Soviet Union'
    }
    
    # URL de base pour les sports olympiques
    main_url = "https://olympics-statistics.com/olympic-sports"
    
    try:
        # Récupérer la page principale des sports
        response = requests.get(main_url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extraire les liens vers les sports
        sport_links = soup.select('a.card.sport.visible')
        
        print(f"Nombre de sports trouvés: {len(sport_links)}")
        
        sports_data = []
        for i, sport_link in enumerate(sport_links):
            href = sport_link.get('href', '')
            if not href:
                continue
                
            sport_url = f"https://olympics-statistics.com{href}"
            sport_name = sport_link.select_one('div.bez').text.strip() if sport_link.select_one('div.bez') else "Unknown Sport"
            
            print(f"Traitement du sport {i+1}/{len(sport_links)}: {sport_name}")
            
            # Extraire les médailles par nation pour ce sport
            sport_data = extract_medals_by_nation(sport_url, pays_dict)
            sports_data.append(sport_data)
            
            # Afficher un résumé des données extraites
            nation_count = len(sport_data.get('nations', []))
            print(f"  - {nation_count} nations trouvées dans TOP NATIONS pour {sport_name}")
            
            # Liste des trois premières nations pour vérification
            if nation_count > 0:
                top_nations = [nation['nom'] for nation in sport_data.get('nations', [])[:3]]
                print(f"  - Top nations: {', '.join(top_nations)}")
            
            # Ajout d'un délai pour éviter de surcharger le serveur
            time.sleep(1)
        
        # Sauvegarder dans un fichier JSON
        output_file = "sports_medals.json"
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(sports_data, f, ensure_ascii=False, indent=4)
        
        print(f"Informations sauvegardées pour {len(sports_data)} sports dans {output_file}")
        
    except Exception as e:
        print(f"Erreur lors de l'exécution du script: {str(e)}")

if __name__ == "__main__":
    main()

Nombre de sports trouvés: 68
Traitement du sport 1/68: Aeronautics
Nombre de nations trouvées pour Aeronautics: 1
  - 1 nations trouvées dans TOP NATIONS pour Aeronautics
  - Top nations: Switzerland
1
Traitement du sport 2/68: Alpine Skiing
Nombre de nations trouvées pour Alpine Skiing: 8
  - 8 nations trouvées dans TOP NATIONS pour Alpine Skiing
  - Top nations: Austria
128, Switzerland
75, United States of America
48
Traitement du sport 3/68: Alpinism
Nombre de nations trouvées pour Alpinism: 3
  - 3 nations trouvées dans TOP NATIONS pour Alpinism
  - Top nations: Germany
1, Great Britain / Australia
1, Switzerland
1
Traitement du sport 4/68: Archery
Nombre de nations trouvées pour Archery: 8
  - 8 nations trouvées dans TOP NATIONS pour Archery
  - Top nations: South Korea
50, United States of America
36, Belgium
21
Traitement du sport 5/68: Athletics


KeyboardInterrupt: 

In [1]:
import json
import pandas as pd
import re

# Chargement des données
try:
    with open('sports_medals.json', 'r', encoding='utf-8') as f:
        sports_data = json.load(f)
    print(f"Données chargées : {len(sports_data)} sports trouvés")
# except FileNotFoundError:
#     print("Fichier sports_medals.json introuvable. Exécutez d'abord le script principal.")
#     sports_data = []

# Dictionnaire simplifié pour les codes pays
pays_dict = {
    'afghanistan': 'af', 'albania': 'al', 'algeria': 'dz', 'argentina': 'ar',
    'armenia': 'am', 'australia': 'au', 'austria': 'at', 'azerbaijan': 'az',
    'bahamas': 'bs', 'belarus': 'by', 'belgium': 'be', 'brazil': 'br',
    'bulgaria': 'bg', 'cameroon': 'cm', 'canada': 'ca', 'chile': 'cl',
    'china': 'cn', 'chinese taipei': 'tw', 'colombia': 'co', 'croatia': 'hr',
    'cuba': 'cu', 'czech republic': 'cz', 'denmark': 'dk', 'egypt': 'eg',
    'estonia': 'ee', 'ethiopia': 'et', 'finland': 'fi', 'france': 'fr',
    'georgia': 'ge', 'germany': 'de', 'great britain': 'gb', 'greece': 'gr',
    'hungary': 'hu', 'india': 'in', 'indonesia': 'id', 'iran': 'ir',
    'ireland': 'ie', 'israel': 'il', 'italy': 'it', 'jamaica': 'jm',
    'japan': 'jp', 'kazakhstan': 'kz', 'kenya': 'ke', 'latvia': 'lv',
    'lithuania': 'lt', 'malaysia': 'my', 'mexico': 'mx', 'mongolia': 'mn',
    'morocco': 'ma', 'netherlands': 'nl', 'new zealand': 'nz', 'nigeria': 'ng',
    'north korea': 'kp', 'norway': 'no', 'pakistan': 'pk', 'poland': 'pl',
    'portugal': 'pt', 'romania': 'ro', 'russia': 'ru', 'serbia': 'rs',
    'slovakia': 'sk', 'slovenia': 'si', 'south africa': 'za', 'south korea': 'kr',
    'soviet union': 'su', 'spain': 'es', 'sweden': 'se', 'switzerland': 'ch',
    'thailand': 'th', 'turkey': 'tr', 'ukraine': 'ua', 'united states': 'us', 
    'united states of america': 'us', 'uzbekistan': 'uz', 'yugoslavia': 'yu'
}

def nettoyer_data_conserver_tous():
    """
    Cette fonction conserve EXACTEMENT tous les sports originaux
    et modifie uniquement le format interne des données.
    """
    # Liste pour stocker les sports nettoyés
    sports_propres = []
    
    # Parcourir exactement tous les sports originaux
    for index, sport_orig in enumerate(sports_data):
        # Extraire le nom du sport
        sport_name = sport_orig.get('sport', f"Sport inconnu {index}")
        
        # Nettoyer légèrement le nom si nécessaire
        if sport_name.startswith("Olympic Medals in Sport "):
            sport_name = sport_name[24:].strip()
            
        # Liste pour stocker les nations nettoyées
        nations_propres = []
        
        # Traiter les nations si elles existent
        nations_orig = sport_orig.get('nations', [])
        for nation in nations_orig:
            # Extraire le nom de la nation
            nation_name_raw = nation.get('nom', '').strip()
            
            # Traiter le format potentiel "Nombre, Pays"
            match = re.match(r'^(\d+),\s*(.+)$', nation_name_raw)
            if match:
                medal_count = int(match.group(1))
                nation_name = match.group(2).strip().title()
            else:
                nation_name = nation_name_raw.title() if nation_name_raw else ""
                medal_count = 0
            
            # Si pas de nom valide, passer à la suite
            if not nation_name:
                continue
                
            # Extraire les médailles
            medailles = nation.get('medailles', {})
            or_medals = medailles.get('or', 0)
            argent_medals = medailles.get('argent', 0)
            bronze_medals = medailles.get('bronze', 0)
            total_medals = medailles.get('total', 0)
            
            # Si le total vient du format "Nombre, Pays"
            if match and total_medals == 0:
                total_medals = medal_count
                
            # Si toujours pas de total mais des médailles par type
            if total_medals == 0 and (or_medals > 0 or argent_medals > 0 or bronze_medals > 0):
                total_medals = or_medals + argent_medals + bronze_medals
            
            # Trouver le code pays
            pays_code = ""
            nom_lower = nation_name.lower()
            if nom_lower in pays_dict:
                pays_code = pays_dict[nom_lower]
            elif 'united states' in nom_lower:
                pays_code = 'us'
            elif 'great britain' in nom_lower:
                pays_code = 'gb'
            
            # Créer la nation propre
            nation_propre = {
                'nom': nation_name,
                'pays_code': pays_code,
                'or': or_medals,
                'argent': argent_medals,
                'bronze': bronze_medals,
                'total': total_medals
            }
            
            nations_propres.append(nation_propre)
        
        # Trier les nations par médailles
        nations_propres = sorted(nations_propres, key=lambda x: x['total'], reverse=True)
        
        # Créer le sport propre
        sport_propre = {
            'nom': sport_name,
            'nombre_nations': len(nations_propres),
            'nations': nations_propres
        }
        
        sports_propres.append(sport_propre)
    
    # Afficher les statistiques
    print(f"Nombre de sports originaux: {len(sports_data)}")
    print(f"Nombre de sports après nettoyage: {len(sports_propres)}")
    
    # Lister les sports pour vérification
    print("\nListe complète des sports traités:")
    for i, sport in enumerate(sports_propres):
        print(f"{i+1}. {sport['nom']} - {sport['nombre_nations']} nations")
    
    # Enregistrer le fichier JSON
    with open('sports_medals_clean.json', 'w', encoding='utf-8') as f:
        json.dump(sports_propres, f, ensure_ascii=False, indent=4)
    
    print(f"\nFichier 'sports_medals_clean.json' créé avec {len(sports_propres)} sports.")
    
    # Vérification finale du fichier JSON
#     try:
#         with open('sports_medals_clean.json', 'r', encoding='utf-8') as f:
#             verify = json.load(f)
#         print(f"Vérification: Le fichier contient bien {len(verify)} sports.")
#     except Exception as e:
#         print(f"Erreur lors de la vérification: {str(e)}")
    
    return sports_propres

# Exécuter le nettoyage qui préserve tous les sports originaux
sports_clean = nettoyer_data_conserver_tous()

Données chargées : 68 sports trouvés
Nombre de sports originaux: 68
Nombre de sports après nettoyage: 68

Liste complète des sports traités:
1. Aeronautics - 1 nations
2. Alpine Skiing - 8 nations
3. Alpinism - 3 nations
4. Archery - 8 nations
5. Athletics - 8 nations
6. Badminton - 8 nations
7. Baseball - 7 nations
8. Basketball - 8 nations
9. Beachvolleyball - 8 nations
10. Biathlon - 8 nations
11. Bobsledding - 8 nations
12. Boxing - 8 nations
13. Breaking - 6 nations
14. Canoe & Kayaking - 8 nations
15. Cricket - 2 nations
16. Crocket - 1 nations
17. Cross-Country Skiing - 8 nations
18. Curling - 8 nations
19. Cycling - 8 nations
20. Diving - 8 nations
21. Equestrian - 8 nations
22. Fencing - 8 nations
23. Figure Skating - 8 nations
24. Football - 8 nations
25. Freestyle Skiing - 8 nations
26. Golf - 8 nations
27. Gymnastics - 8 nations
28. Handball - 8 nations
29. Hockey - 8 nations
30. Ice Hockey - 8 nations
31. Jeu de Paume - 2 nations
32. Judo - 8 nations
33. Karate - 8 nations

In [2]:
from bs4 import BeautifulSoup
import requests
import json
import re
import time

def extract_medals_by_nation(sport_url, pays_dict):
    try:
        response = requests.get(sport_url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        sport_name = "Unknown Sport"
        sport_title = soup.select_one('h1')
        if sport_title:
            sport_name = sport_title.text.strip().replace("Olympic Medals in Sport ", "")
        else:
            match = re.search(r'/olympic-sport/([^/]+)', sport_url)
            if match:
                sport_name = match.group(1).replace('-', ' ')
        
        nation_cards = soup.select('div.cards div.card.nation.visible') or soup.select('div.deck div.card.nation.visible')
        print(f"Nombre de nations trouvées pour {sport_name}: {len(nation_cards)}")
        
        nations = []
        for nation_card in nation_cards:
            name_elem = nation_card.select_one('div.bez')
            if not name_elem:
                continue
                
            nation_name = name_elem.text.strip()
            
            medals_elem = nation_card.select_one('div.mal-ges')
            total_medals = 0
            if medals_elem:
                medals_match = re.search(r'(\d+)', medals_elem.text.strip())
                if medals_match:
                    total_medals = int(medals_match.group(1))
            
            medal_counts = {'or': 0, 'argent': 0, 'bronze': 0}
            medal_type_map = {'1': 'or', '2': 'argent', '3': 'bronze'}
            
            for medal_elem in nation_card.select('div.medals div[data-c]'):
                medal_count_elem = medal_elem.select_one('span.mal')
                if not medal_count_elem:
                    continue
                
                medal_count = medal_count_elem.text.strip()
                if not medal_count.isdigit():
                    continue
                    
                medal_type_elem = medal_elem.select_one('div.the-medal')
                if medal_type_elem:
                    medal_type = medal_type_elem.get('data-medal')
                    if medal_type in medal_type_map:
                        medal_counts[medal_type_map[medal_type]] = int(medal_count)
            
            nation = {
                "nom": nation_name,
                "medailles": {
                    "total": total_medals,
                    **medal_counts
                }
            }
            
            nation_name_lower = nation_name.lower()
            for code, name in pays_dict.items():
                if name.lower() == nation_name_lower:
                    nation["code_pays"] = code
                    break
            
            nations.append(nation)
        
        return {"sport": sport_name, "nations": nations}
        
    except Exception as e:
        print(f"Erreur pour {sport_url}: {str(e)}")
        return {"sport": sport_url, "nations": [], "error": str(e)}

def main():
    pays_dict = {
        'us': 'United States', 'fr': 'France', 'fi': 'Finland', 'de': 'Germany',
        'it': 'Italy', 'gb': 'Great Britain', 'uk': 'United Kingdom', 'ca': 'Canada',
        # Ajoutez tous les autres pays ici
        'urs': 'Soviet Union'
    }
    
    try:
        response = requests.get("https://olympics-statistics.com/olympic-sports")
        soup = BeautifulSoup(response.text, 'html.parser')
        sport_links = soup.select('a.card.sport.visible')
        
        print(f"Nombre de sports trouvés: {len(sport_links)}")
        
        sports_data = []
        for i, sport_link in enumerate(sport_links):
            href = sport_link.get('href')
            if not href:
                continue
                
            sport_url = f"https://olympics-statistics.com{href}"
            sport_name = sport_link.select_one('div.bez').text.strip() if sport_link.select_one('div.bez') else "Unknown Sport"
            
            print(f"Traitement du sport {i+1}/{len(sport_links)}: {sport_name}")
            
            sport_data = extract_medals_by_nation(sport_url, pays_dict)
            sports_data.append(sport_data)
            
            nations = sport_data.get('nations', [])
            print(f"  - {len(nations)} nations trouvées pour {sport_name}")
            
            if nations:
                top_nations = [nation['nom'] for nation in nations[:3]]
                print(f"  - Top nations: {', '.join(top_nations)}")
            
            time.sleep(0.5)
        
        with open("sports_medals.json", 'w', encoding='utf-8') as f:
            json.dump(sports_data, f, ensure_ascii=False, indent=4)
        
        print(f"Informations sauvegardées pour {len(sports_data)} sports")
        
    except Exception as e:
        print(f"Erreur d'exécution: {str(e)}")

if __name__ == "__main__":
    main()

Nombre de sports trouvés: 68
Traitement du sport 1/68: Aeronautics
Nombre de nations trouvées pour Aeronautics: 1
  - 1 nations trouvées pour Aeronautics
  - Top nations: Switzerland
1
Traitement du sport 2/68: Alpine Skiing
Nombre de nations trouvées pour Alpine Skiing: 8
  - 8 nations trouvées pour Alpine Skiing
  - Top nations: Austria
128, Switzerland
75, United States of America
48
Traitement du sport 3/68: Alpinism
Nombre de nations trouvées pour Alpinism: 3
  - 3 nations trouvées pour Alpinism
  - Top nations: Germany
1, Great Britain / Australia
1, Switzerland
1
Traitement du sport 4/68: Archery
Nombre de nations trouvées pour Archery: 8
  - 8 nations trouvées pour Archery
  - Top nations: South Korea
50, United States of America
36, Belgium
21
Traitement du sport 5/68: Athletics
Nombre de nations trouvées pour Athletics: 8
  - 8 nations trouvées pour Athletics
  - Top nations: United States of America
884, Russia
273, Germany
271
Traitement du sport 6/68: Badminton
Nombre de n

In [5]:
import json
import pandas as pd
import re

try:
    with open('sports_medals.json', 'r', encoding='utf-8') as f:
        sports_data = json.load(f)
    print(f"Données chargées : {len(sports_data)} sports trouvés")
except FileNotFoundError:
    print("Fichier sports_medals.json introuvable")
    sports_data = []

pays_dict = {
    'afghanistan': 'af', 'albania': 'al', 'algeria': 'dz', 'argentina': 'ar',
    'armenia': 'am', 'australia': 'au', 'austria': 'at', 'azerbaijan': 'az',
    'bahamas': 'bs', 'belarus': 'by', 'belgium': 'be', 'brazil': 'br',
    'bulgaria': 'bg', 'cameroon': 'cm', 'canada': 'ca', 'chile': 'cl',
    'china': 'cn', 'chinese taipei': 'tw', 'colombia': 'co', 'croatia': 'hr',
    'cuba': 'cu', 'czech republic': 'cz', 'denmark': 'dk', 'egypt': 'eg',
    'estonia': 'ee', 'ethiopia': 'et', 'finland': 'fi', 'france': 'fr',
    'georgia': 'ge', 'germany': 'de', 'great britain': 'gb', 'greece': 'gr',
    'hungary': 'hu', 'india': 'in', 'indonesia': 'id', 'iran': 'ir',
    'ireland': 'ie', 'israel': 'il', 'italy': 'it', 'jamaica': 'jm',
    'japan': 'jp', 'kazakhstan': 'kz', 'kenya': 'ke', 'latvia': 'lv',
    'lithuania': 'lt', 'malaysia': 'my', 'mexico': 'mx', 'mongolia': 'mn',
    'morocco': 'ma', 'netherlands': 'nl', 'new zealand': 'nz', 'nigeria': 'ng',
    'north korea': 'kp', 'norway': 'no', 'pakistan': 'pk', 'poland': 'pl',
    'portugal': 'pt', 'romania': 'ro', 'russia': 'ru', 'serbia': 'rs',
    'slovakia': 'sk', 'slovenia': 'si', 'south africa': 'za', 'south korea': 'kr',
    'soviet union': 'su', 'spain': 'es', 'sweden': 'se', 'switzerland': 'ch',
    'thailand': 'th', 'turkey': 'tr', 'ukraine': 'ua', 'united states': 'us', 
    'united states of america': 'us', 'uzbekistan': 'uz', 'yugoslavia': 'yu'
}

def nettoyer_data_conserver_tous():
    sports_propres = []
    
    for index, sport_orig in enumerate(sports_data):
        sport_name = sport_orig.get('sport', f"Sport inconnu {index}")
        
        if sport_name.startswith("Olympic Medals in Sport "):
            sport_name = sport_name[24:].strip()
            
        nations_propres = []
        
        nations_orig = sport_orig.get('nations', [])
        for nation in nations_orig:
            nation_name_raw = nation.get('nom', '').strip()
            
            match = re.match(r'^(\d+),\s*(.+)$', nation_name_raw)
            if match:
                medal_count = int(match.group(1))
                nation_name = match.group(2).strip().title()
            else:
                nation_name = nation_name_raw.title() if nation_name_raw else ""
                medal_count = 0
            
            if not nation_name:
                continue
                
            medailles = nation.get('medailles', {})
            or_medals = medailles.get('or', 0)
            argent_medals = medailles.get('argent', 0)
            bronze_medals = medailles.get('bronze', 0)
            total_medals = medailles.get('total', 0)
            
            if match and total_medals == 0:
                total_medals = medal_count
                
            if total_medals == 0 and (or_medals > 0 or argent_medals > 0 or bronze_medals > 0):
                total_medals = or_medals + argent_medals + bronze_medals
            
            pays_code = ""
            nom_lower = nation_name.lower()
            if nom_lower in pays_dict:
                pays_code = pays_dict[nom_lower]
            elif 'united states' in nom_lower:
                pays_code = 'us'
            elif 'great britain' in nom_lower:
                pays_code = 'gb'
            
            nation_propre = {
                'nom': nation_name,
                'pays_code': pays_code,
                'or': or_medals,
                'argent': argent_medals,
                'bronze': bronze_medals,
                'total': total_medals
            }
            
            nations_propres.append(nation_propre)
        
        nations_propres = sorted(nations_propres, key=lambda x: x['total'], reverse=True)
        
        sport_propre = {
            'nom': sport_name,
            'nombre_nations': len(nations_propres),
            'nations': nations_propres
        }
        
        sports_propres.append(sport_propre)
    
    print(f"Nombre de sports originaux: {len(sports_data)}")
    print(f"Nombre de sports après nettoyage: {len(sports_propres)}")
    
    print("\nListe complète des sports traités:")
    for i, sport in enumerate(sports_propres):
        print(f"{i+1}. {sport['nom']} - {sport['nombre_nations']} nations")
    
    with open('sports_medals_clean.json', 'w', encoding='utf-8') as f:
        json.dump(sports_propres, f, ensure_ascii=False, indent=4)
    
    print(f"\nFichier 'sports_medals_clean.json' créé avec {len(sports_propres)} sports.")
    
    return sports_propres

sports_clean = nettoyer_data_conserver_tous()

Données chargées : 68 sports trouvés
Nombre de sports originaux: 68
Nombre de sports après nettoyage: 68

Liste complète des sports traités:
1. Aeronautics - 1 nations
2. Alpine Skiing - 8 nations
3. Alpinism - 3 nations
4. Archery - 8 nations
5. Athletics - 8 nations
6. Badminton - 8 nations
7. Baseball - 7 nations
8. Basketball - 8 nations
9. Beachvolleyball - 8 nations
10. Biathlon - 8 nations
11. Bobsledding - 8 nations
12. Boxing - 8 nations
13. Breaking - 6 nations
14. Canoe & Kayaking - 8 nations
15. Cricket - 2 nations
16. Crocket - 1 nations
17. Cross-Country Skiing - 8 nations
18. Curling - 8 nations
19. Cycling - 8 nations
20. Diving - 8 nations
21. Equestrian - 8 nations
22. Fencing - 8 nations
23. Figure Skating - 8 nations
24. Football - 8 nations
25. Freestyle Skiing - 8 nations
26. Golf - 8 nations
27. Gymnastics - 8 nations
28. Handball - 8 nations
29. Hockey - 8 nations
30. Ice Hockey - 8 nations
31. Jeu de Paume - 2 nations
32. Judo - 8 nations
33. Karate - 8 nations